<h1><center> Generate macromolecules surface concentration for ICON model input </center></h1>

<h4><center> Author: Anisbel León Marcos${^1}$ </center></h4>
<h6><center> ${^1}$Institute for Tropospheric Research (TROPOS)
 ${^1}$leon@tropos.de  
</center></h6>

<br/>

In [1]:
import numpy as np
from netCDF4 import Dataset
import datetime as dt
from netCDF4 import date2num,num2date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy as cart


In [2]:
path = './'
# temp_global_CapeVerde
file_name = 'monthly_macromolecules_0.1deg_bilinear_latlon_year01_merge_date.nc'

l = 1                                  # to plot other variables used l = 1, unless you use the TEP original nc file
nc = Dataset(path+file_name,mode='r')

# print(nc)

lat_0 = np.array(nc.variables['lat'][:])
lon_0 = np.array(nc.variables['lon'][:])
time_0 = np.array(nc.variables['time'][:])

var = ['TRUEPROTC','TRUEPOLYC','TRUELIPC','TRUEHUMC','TRUEPROCC','CHL1']



In [3]:
C = []
[C.append(np.array(nc.variables[var[i]][:][:])) for i in range(len(var))]

a,b,c = C[0].shape

C_tot_new = []
C_new =  [np.zeros((a,b,c)) for i in range(len(var))]
print(a,b,c,len(C_new[0][0]))

for i in range(len(var)):
    for l in range(len(C[i])):
        C_new[i][l] = C[i][l-1]  
time_0



12 1800 3600 1800


array([388., 416., 447., 477., 508., 538., 569., 600., 630., 661., 691.,
       722.])

In [4]:
file_name = 'TRUELIPC.nc'

l = 1                                  # to plot other variables used l = 1, unless you use the TEP original nc file
nc = Dataset(path+file_name,mode='r')

# print(nc)

lat_0 = np.array(nc.variables['lat'][:])
lon_0 = np.array(nc.variables['lon'][:])
time_0 = np.array(nc.variables['time'][:])

C_lip = np.array(nc.variables['TRUELIPC'][:][:])


In [9]:
C_lip.max()

16.65902914362495

In [ ]:
for i in range(len(var)):
    try: ncfile.close()  # just to be safe, make sure dataset is not already open.
    except: pass

    ncfile = Dataset(var[i]+'.nc',mode='w',format='NETCDF4_CLASSIC') 
    ncfile.title=var[i]


    lat_dim = ncfile.createDimension('lat', b)     # latitude axis
    lon_dim = ncfile.createDimension('lon', c)    # longitude axis
    time_dim = ncfile.createDimension('time',a) # unlimited axis (can be appended to).

    lat = ncfile.createVariable('lat', np.float32, ('lat',))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lon = ncfile.createVariable('lon', np.float32, ('lon',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    time = ncfile.createVariable('time', np.float64, ('time',))
    time.units = 'days since 2013-01-01 00:00:00'
    time.long_name = 'time'
    # Define a 3D variable to hold the data
    var_nc = ncfile.createVariable(var[i],np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost
    var_nc.units = "uMol C" 
    var_nc.standard_name = 'Surface ocean concentration of ' + var[i] 

    dates = [dt.datetime(2013,n+1,1,0) for n in range(12)]
    #pd.date_range(start='01/01/2013', periods=12, freq='M').strftime("%m/%d/%Y, %H:%M:%S").tolist()
#     reference_time = pd.Timestamp("2009-01-01")
    time_n = date2num(dates,units=time.units)
    
    nlats = len(lat_dim); nlons = len(lon_dim)
    # Write latitudes, longitudes.
    # Note: the ":" is necessary in these "write" statements
    lat[:] = lat_0 # south pole to north pole
    lon[:] = lon_0 # Greenwich meridian eastward
    time[:] = time_n
    # Write the data.  This writes the whole 3D netCDF variable all at once.
    var_nc[:,:,:] = C_new[i]  # Appends data along unlimited dimension
#     print(C_new[i].max(),C_new[i].min())
    print(ncfile)
    ncfile.close()

In [ ]:
print(ncfile)

In [ ]:
import xarray as xr
data = xr.open_dataset('TRUEPOLYC.nc') 
data = data.isel(time = 1)
fig, ax = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.PlateCarree())
)

ax.plot(data.lat,data.lon,data)

In [2]:
import xarray as xr
data = xr.open_dataset('TRUEPOLYC.nc')

In [4]:
data.max()

<xarray.Dataset>
Dimensions:    ()
Data variables:
    TRUEPOLYC  float64 88.29

In [6]:
import xarray as xr
data = xr.open_dataset('TRUELIPC.nc')

In [7]:
data.max()

<xarray.Dataset>
Dimensions:   ()
Data variables:
    TRUELIPC  float64 16.66

In [ ]:
0.6011677984327447
0.6011677984327447